<a href="https://colab.research.google.com/github/nilakshi104/Steering-angle-prediction/blob/master/Steering_angle_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
#path for image='/content/drive/My Drive/ADAS1/IMG'
#path for csv file='/content/drive/My Drive/ADAS1/driving_log.csv'

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import scipy
from scipy import signal
import pandas as pd 
import torch
from torch.utils import data
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import torchvision.transforms as transforms
import numpy as np
import os
from torch.optim.lr_scheduler import MultiStepLR

dataroot = '/content/drive/My Drive/Untitled folder/steer_pred/IMG/'
dataroot1='/content/drive/My Drive/Untitled folder/steer_pred/'
ckptroot = '/content/drive/My Drive/Untitled folder/steer_pred'

#hyperparameters
lr = 1e-4
weight_decay = 1e-5
batch_size = 32
num_workers = 2
train_size = 0.8
shuffle = True

epochs = 200
start_epoch = 0
resume = False

# def toDevice(datas, device):
#     """Enable cuda."""
#     imgs, angles = datas
#     return imgs.float().to(device), angles.float().to(device)
#     # /content/drive/My Drive/ADAS1/driving_log.csv.gsheet/content/drive/My Drive/ADAS1/driving_log.gsheet

In [ ]:

def load_data(data_dir,test_size):
  data_df=pd.read_csv(os.path.join(data_dir,'driving_log.csv'),names=['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed'])
   
  data_df["steering"] = signal.savgol_filter(data_df["steering"].values.tolist(), 51, 11)

  # Divide the data into training set and validation set
  train_len = int(train_size * data_df.shape[0])
  valid_len = data_df.shape[0] - train_len
  trainset, valset = data.random_split(
      data_df.values.tolist(), lengths=[train_len, valid_len])

  return trainset, valset

trainset, valset = load_data(dataroot1, train_size)


In [ ]:
trainset[0][3]

-0.17207182919064376

In [ ]:
def augment(dataroot,imgName, angle):
  name = dataroot + 'IMG/'+ imgName.split('\\')[-1]
  current_image = cv2.imread(name)
  if current_image is None:
    print('image is not available ')
  current_image = current_image[65:-25, :, :]
  if np.random.rand() < 0.5:
    current_image = cv2.flip(current_image, 1)
    angle = angle * -1.0
  return current_image, angle

In [ ]:
k=cv2.imread('/content/drive/My Drive/Untitled folder/steer_pred/IMG/center_2020_01_11_21_16_41_804.jpg')
k = k[65:-25, :, :]
k.shape

(70, 320, 3)

In [ ]:
training_set=[]
for index in range(0,len(trainset)):
  list1=[]
  batch_samples = trainset[index]
  steering_angle = float(batch_samples[3])

  center_img, steering_angle_center = augment(dataroot1, batch_samples[0], steering_angle )
  left_img, steering_angle_left     = augment (dataroot1 ,batch_samples[1], steering_angle+0.4)
  right_img, steering_angle_right   =  augment (dataroot1, batch_samples[2], steering_angle-0.4)

  list1.append((center_img, steering_angle_center))
  list1.append((left_img, steering_angle_left))
  list1.append((right_img, steering_angle_right))
  training_set.append(list1)
# training_set

validation_set=[]
for index in range(0,len(valset)):
  list2=[]
  batch_samples = valset[index]
  steering_angle = float(batch_samples[3])

  center_img, steering_angle_center = augment(dataroot1, batch_samples[0], steering_angle )
  left_img, steering_angle_left     = augment (dataroot1 ,batch_samples[1], steering_angle+0.4)
  right_img, steering_angle_right   =  augment (dataroot1, batch_samples[2], steering_angle-0.4)

  list2.append((center_img, steering_angle_center))
  list2.append((left_img, steering_angle_left))
  list2.append((right_img, steering_angle_right))
  validation_set.append(list2)
# validation_set


def toDevice(datas, device):
    """Enable cuda."""
    imgs, angles = datas
    return imgs.float().to(device), angles.float().to(device)

In [ ]:
print("==> Preparing dataset ...")
def data_loader(training_set, validation_set, batch_size, shuffle, num_workers):
    transformations = transforms.Compose([
           transforms.ToTensor(),
           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    # Load training data and validation data
    trainloader = DataLoader(training_set,
                             batch_size=batch_size,
                             shuffle=shuffle,
                             num_workers=num_workers)

    # validation_set = TripletDataset(dataroot, valset, transformations)
    valloader = DataLoader(validation_set,
                           batch_size=batch_size,
                           shuffle=shuffle,
                           num_workers=num_workers)

    return trainloader, valloader


trainloader, validationloader= data_loader( training_set, validation_set,
                                            batch_size,
                                            shuffle,
                                            num_workers)

==> Preparing dataset ...


In [ ]:
class NetworkNvidia(nn.Module):
    """NVIDIA model used in the paper."""

    def __init__(self):
        """Initialize NVIDIA model.

        NVIDIA model used
            Image normalization to avoid saturation and make gradients work better.
            Convolution: 5x5, filter: 24, strides: 2x2, activation: ELU
            Convolution: 5x5, filter: 36, strides: 2x2, activation: ELU
            Convolution: 5x5, filter: 48, strides: 2x2, activation: ELU
            Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
            Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
            Drop out (0.5)
            Fully connected: neurons: 100, activation: ELU
            Fully connected: neurons: 50, activation: ELU
            Fully connected: neurons: 10, activation: ELU
            Fully connected: neurons: 1 (output)

        the convolution layers are meant to handle feature engineering
        the fully connected layer for predicting the steering angle.
        """
        super(NetworkNvidia, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 24, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(24, 36, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(36, 48, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(48, 64, 3),
            nn.ELU(),
            nn.Conv2d(64, 64, 3),
            nn.Dropout(0.5)
        )
        self.linear_layers = nn.Sequential(
            nn.Linear(in_features=64 * 2 * 33, out_features=100),
            nn.ELU(),
            nn.Linear(in_features=100, out_features=50),
            nn.ELU(),
            nn.Linear(in_features=50, out_features=10),
            nn.Linear(in_features=10, out_features=1)
        )

    def forward(self, input):
        """Forward pass."""
        input = input.view(input.size(0), 3, 70, 320)
        output = self.conv_layers(input)
        # print(output.shape)
        output = output.view(output.size(0), -1)
        output = self.linear_layers(output)
        return output


# Define model
print("==> Initialize model ...")

model = NetworkNvidia()

print("==> Initialize model done ...")


==> Initialize model ...
==> Initialize model done ...


In [ ]:
# Define optimizer and criterion
optimizer = optim.Adam(model.parameters(),
                       lr=lr,
                       weight_decay=weight_decay)
criterion = nn.MSELoss()
scheduler = MultiStepLR(optimizer, milestones=[30, 50], gamma=0.1)

# transfer to gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# for local_batch, (centers, lefts, rights) in enumerate(trainloader):
#   if local_batch==0:
#     datas = [centers, lefts, rights]
#     for data in datas:
#       imgs, angles = data
#       # print(imgs)
#       # print(angles.unsqueeze(1))
#       outputs=model(imgs.float())
#       ans=criterion(outputs,angles.unsqueeze(1).float())
#       print(ans)


In [ ]:
class Trainer(object):
    """Trainer."""

    def __init__(self,
                 ckptroot,
                 model,
                #  device,
                 epochs,
                 criterion,
                 optimizer,
                 scheduler,
                 start_epoch,
                 trainloader,
                 validationloader):
        """Self-Driving car Trainer.

        Args:
            model:
            device:
            epochs:
            criterion:
            optimizer:
            start_epoch:
            trainloader:
            validationloader:

        """
        super(Trainer, self).__init__()

        self.model = model
        # self.device = device
        self.epochs = epochs
        self.ckptroot = ckptroot
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.start_epoch = start_epoch
        self.trainloader = trainloader
        self.validationloader = validationloader

    def train(self):
        """Training process."""
        # self.model.to(self.device)
        for epoch in range(self.start_epoch, self.epochs + self.start_epoch):
            # Training
            train_loss = 0.0
            self.model.train()
            for local_batch, (centers, lefts, rights) in enumerate(self.trainloader):
                # centers, lefts, rights = toDevice(centers, self.device), toDevice(
                #     lefts, self.device), toDevice(rights, self.device)
                self.optimizer.zero_grad()
                datas = [centers, lefts, rights]
                for data in datas:
                    imgs, angles = data
                    # print("training image: ", imgs.shape)
                    outputs = self.model(imgs)
                    ## outpts=outputs.type(torch.LongTensor)
                    loss = self.criterion(outputs, angles.unsqueeze(1).float())
                    loss.backward()
                    self.optimizer.step()
                    train_loss += loss
                if local_batch % 100 == 0:
                  print("Training Epoch: {} | Loss: {}".format(epoch, train_loss / (local_batch + 1)))

            self.scheduler.step()

            # Validation
            self.model.eval()
            valid_loss = 0
            with torch.set_grad_enabled(False):
                for local_batch, (centers, lefts, rights) in enumerate(self.validationloader):
                    # centers, lefts, rights = toDevice(centers, self.device), toDevice(
                    #     lefts, self.device), toDevice(rights, self.device)
                    self.optimizer.zero_grad()
                    datas = [centers, lefts, rights]
                    for data in datas:
                        imgs, angles = data
                        outputs = self.model(imgs)
                        loss = self.criterion(outputs, angles.unsqueeze(1).float())
                        valid_loss += loss

                    if local_batch % 100 == 0:
                      print("Validation Loss: {}".format(valid_loss / (local_batch + 1)))

            print()
            # Save model
            if epoch >=198:

                state = {
                    'epoch': epoch + 1,
                    'state_dict': self.model.state_dict(),
                    'optimizer': self.optimizer.state_dict(),
                    'scheduler': self.scheduler.state_dict(),
                }

                self.save_checkpoint(state)

    def save_checkpoint(self, state):
        """Save checkpoint."""
        print("==> Save checkpoint ...")
        if not os.path.exists(self.ckptroot):
            os.makedirs(self.ckptroot)

        torch.save(state, self.ckptroot + 'steer_pred_model-{}.h5'.format(state['epoch']))
           

In [ ]:
print("==> Start training ...")
trainer = Trainer(ckptroot,
                  model,
                  # device,
                  epochs,
                  criterion,
                  optimizer,
                  scheduler,
                  start_epoch,
                  trainloader,
                  validationloader)
trainer.train()

==> Start training ...
Training Epoch: 0 | Loss: 7.29891107766889e-05
Validation Loss: 0.21145768160931766

==> Save checkpoint ...
Training Epoch: 1 | Loss: 0.00010292530532751698
Validation Loss: 0.21817972871940583

Training Epoch: 2 | Loss: 0.0001275957747566281
Validation Loss: 0.22101195470895618

Training Epoch: 3 | Loss: 0.00011032108341169078
Validation Loss: 0.21697034465614706

Training Epoch: 4 | Loss: 0.00011835017721750773
Validation Loss: 0.22135543334297836

Training Epoch: 5 | Loss: 7.824223530406016e-05
Validation Loss: 0.21477255527861416

==> Save checkpoint ...
Training Epoch: 6 | Loss: 0.00015936781346681528
Validation Loss: 0.21711772633716464

Training Epoch: 7 | Loss: 0.00011791492124757497
Validation Loss: 0.21643018745817244

Training Epoch: 8 | Loss: 7.58652913646074e-05
Validation Loss: 0.20645950292237103

Training Epoch: 9 | Loss: 9.327893803856568e-05
Validation Loss: 0.21724949800409377

Training Epoch: 10 | Loss: 8.44368396428763e-05
Validation Loss: 0

In [ ]:

# class TripletDataset(data.Dataset):

#     def __init__(self, data_root, samples, transform=None):
#         self.samples = samples
#         self.data_root = data_root
#         self.transform = transform

#     # def augment(self,imgName, angle):
#     #     name = self.data_root + 'IMG/'+ imgName.split('\\')[-1]
#     #     current_image = cv2.imread(name)
#     #     if current_image is None:
#     #       print('image is not available ')
#     #     return current_image, angle

#     def hh( self,index):
#         batch_samples = self.samples[index]
#         steering_angle = float(batch_samples[3])

#         center_img, steering_angle_center =  batch_samples[0], steering_angle
#         left_img, steering_angle_left     =  batch_samples[1], steering_angle+0.4
#         right_img, steering_angle_right   =  batch_samples[2], steering_angle-0.4)

#         # center_img = self.transform(center_img)
#         # left_img   = self.transform(left_img)
#         # right_img  = self.transform(right_img)

#         return  (steering_angle_center), (steering_angle_left), (steering_angle_right)
#         # return 1

#     def __len__(self):
#         return len(self.samples)